In [2]:
!pip install mne -q

import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, roc_curve, auc
from mlxtend.plotting import plot_confusion_matrix
import scipy.stats as stats

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [5]:
krisa1_path = "/kaggle/input/krisii/Ati4x1_15m_BL_6h_fully_marked.edf"
krisa2_path = "/kaggle/input/krisii/Ati4x1_15m_Dex003(Pharm)_6h_fully_marked.edf"
krisa3_path = "/kaggle/input/krisii/Ati4x1_15m_H2O_6h_fully_marked.edf"
krisa4_path = "/kaggle/input/krisii/Ati4x3_12m_BL_6h_fully_marked.edf"
krisa5_path = "/kaggle/input/krisii/Ati4x3_9m_Xyl01(Pharm)_6h_fully_marked.edf"
krisa6_path = "/kaggle/input/krisii/Ati4x6_14m_BL_6h_fully_marked.edf"

krisa1_data = mne.io.read_raw_edf(krisa1_path)
krisa2_data = mne.io.read_raw_edf(krisa2_path)
krisa3_data = mne.io.read_raw_edf(krisa3_path)
krisa4_data = mne.io.read_raw_edf(krisa4_path)
krisa5_data = mne.io.read_raw_edf(krisa5_path)
krisa6_data = mne.io.read_raw_edf(krisa6_path)

df1 = krisa1_data.to_data_frame()
df2 = krisa2_data.to_data_frame()
df3 = krisa3_data.to_data_frame()
df4 = krisa4_data.to_data_frame()
df5 = krisa5_data.to_data_frame()
df6 = krisa6_data.to_data_frame()

df1['time'] = pd.to_datetime(df1['time'], unit='s').dt.strftime('%H:%M:%S.%f')
df2['time'] = pd.to_datetime(df2['time'], unit='s').dt.strftime('%H:%M:%S.%f')
df3['time'] = pd.to_datetime(df3['time'], unit='s').dt.strftime('%H:%M:%S.%f')
df4['time'] = pd.to_datetime(df4['time'], unit='s').dt.strftime('%H:%M:%S.%f')
df5['time'] = pd.to_datetime(df5['time'], unit='s').dt.strftime('%H:%M:%S.%f')
df6['time'] = pd.to_datetime(df6['time'], unit='s').dt.strftime('%H:%M:%S.%f')

Extracting EDF parameters from /kaggle/input/krisii/Ati4x1_15m_BL_6h_fully_marked.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /kaggle/input/krisii/Ati4x1_15m_Dex003(Pharm)_6h_fully_marked.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /kaggle/input/krisii/Ati4x1_15m_H2O_6h_fully_marked.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /kaggle/input/krisii/Ati4x3_12m_BL_6h_fully_marked.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /kaggle/input/krisii/Ati4x3_9m_Xyl01(Pharm)_6h_fully_marked.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /kaggle/input/krisii/Ati4x6_14m_BL_6h_fully_marked.edf...
EDF file detected
Setting chan

In [6]:
krisa1_annotations = pd.read_csv('/kaggle/input/krisii/Ati4x1_15m_BL_6h.txt', sep="\t", names=["NN", "time", "marker"], header=0)
krisa2_annotations = pd.read_csv('/kaggle/input/krisii/Ati4x1_15m_Dex003(Pharm)_6h.txt', sep="\t", names=["NN", "time", "marker"], header=0)
krisa3_annotations = pd.read_csv('/kaggle/input/krisii/Ati4x1_15m_H2O_6h.txt', sep="\t", names=["NN", "time", "marker"], header=0)
krisa4_annotations = pd.read_csv('/kaggle/input/krisii/Ati4x3_12m_BL_6h.txt', sep="\t", names=["NN", "time", "marker"], header=0)
krisa5_annotations = pd.read_csv('/kaggle/input/krisii/Ati4x3_9m_Xyl01(Pharm)_6h1.txt', sep="\t", names=["NN", "time", "marker"], header=0)
krisa6_annotations = pd.read_csv('/kaggle/input/krisii/Ati4x6_14m_BL_6h.txt', sep="\t", names=["NN", "time", "marker"], header=0)

krisa1_annotations.drop(columns=['NN'], inplace=True, axis=1)
krisa2_annotations.drop(columns=['NN'], inplace=True, axis=1)
krisa3_annotations.drop(columns=['NN'], inplace=True, axis=1)
krisa4_annotations.drop(columns=['NN'], inplace=True, axis=1)
krisa5_annotations.drop(columns=['NN'], inplace=True, axis=1)
krisa6_annotations.drop(columns=['NN'], inplace=True, axis=1)

krisa1_annotations['time'] = pd.to_datetime(krisa1_annotations['time'], format='%H:%M:%S')
krisa2_annotations['time'] = pd.to_datetime(krisa2_annotations['time'], format='%H:%M:%S')
krisa3_annotations['time'] = pd.to_datetime(krisa3_annotations['time'], format='%H:%M:%S')
krisa4_annotations['time'] = pd.to_datetime(krisa4_annotations['time'], format='%H:%M:%S')
krisa5_annotations['time'] = pd.to_datetime(krisa5_annotations['time'], format='%H:%M:%S')
krisa6_annotations['time'] = pd.to_datetime(krisa6_annotations['time'], format='%H:%M:%S')

krisa1_annotations['time'] = krisa1_annotations['time'].dt.strftime('%H:%M:%S.') + '000000'
krisa2_annotations['time'] = krisa2_annotations['time'].dt.strftime('%H:%M:%S.') + '000000'
krisa3_annotations['time'] = krisa3_annotations['time'].dt.strftime('%H:%M:%S.') + '000000'
krisa4_annotations['time'] = krisa4_annotations['time'].dt.strftime('%H:%M:%S.') + '000000'
krisa5_annotations['time'] = krisa5_annotations['time'].dt.strftime('%H:%M:%S.') + '000000'
krisa6_annotations['time'] = krisa6_annotations['time'].dt.strftime('%H:%M:%S.') + '000000'

In [7]:
df1 = pd.merge(df1, krisa1_annotations, on='time', how='left')
df2 = pd.merge(df2, krisa2_annotations, on='time', how='left')
df3 = pd.merge(df3, krisa3_annotations, on='time', how='left')
df4 = pd.merge(df4, krisa4_annotations, on='time', how='left')
df5 = pd.merge(df5, krisa5_annotations, on='time', how='left')
df6 = pd.merge(df6, krisa6_annotations, on='time', how='left')

In [8]:
df1['krisa'] = 1
df2['krisa'] = 2
df3['krisa'] = 3
df4['krisa'] = 4
df5['krisa'] = 5
df6['krisa'] = 6

df = pd.concat([df1, df2, df3, df4, df5, df6], axis=0, ignore_index=True)

In [9]:
df.shape

(51842407, 6)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51842407 entries, 0 to 51842406
Data columns (total 6 columns):
 #   Column  Dtype  
---  ------  -----  
 0   time    object 
 1   FrL     float64
 2   FrR     float64
 3   OcR     float64
 4   marker  object 
 5   krisa   int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 2.3+ GB


In [ ]:
unique_values = df["marker"].unique()

for value in unique_values:
    print(value)

nan
ds1
ds2
is2
is1
swd1
swd2
dds2
sdw1


In [14]:
# Преобразуем значения
df['marker'] = df['marker'].replace({"sdw1": "swd1", "dds2": "ds2"})

In [ ]:
unique_values = df["marker"].unique()

for value in unique_values:
    print(value)

nan
ds1
ds2
is2
is1
swd1
swd2


In [ ]:
def fill_event_periods(df):
    event_start = None
    event_type = None


    for i, row in df.iterrows():
        if row['marker'] in ['swd1', 'ds1', 'is1']:  # Если находим начало события
            event_start = i
            event_type = row['marker']  # Запоминаем тип события

        elif row['marker'] in ['swd2', 'ds2', 'is2'] and event_start is not None:  # Конец события
            event_end = i
            # Заполняем промежутки между началом и концом события
            df.loc[event_start + 1:event_end - 1, 'marker'] = event_type
            event_start = None  # Сброс
            event_type = None   # Сброс

    df['marker'] = df['marker'].replace({'ds1': 'ds', 'ds2': 'ds', 'is1': 'is', 'is2': 'is', 'swd1': 'swd', 'swd2': 'swd'})
    return df

df_filled = fill_event_periods(df)

df_filled.head()

,time,FrL,FrR,OcR,marker,krisa
0,00:00:00.000000,50.8750,46.5625,121.1250,NaN,1
1,00:00:00.002500,41.8125,36.2500,110.2500,NaN,1
2,00:00:00.005000,51.8125,39.9375,122.8750,NaN,1
3,00:00:00.007500,74.7500,67.9375,150.9375,NaN,1
4,00:00:00.010000,86.8750,86.0625,177.6875,NaN,1


In [19]:
df_filled['marker'].fillna(value='none', inplace=True)

/tmp/ipykernel_30/628020086.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filled['marker'].fillna(value='none', inplace=True)


In [20]:
df_filled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51842407 entries, 0 to 51842406
Data columns (total 6 columns):
 #   Column  Dtype  
---  ------  -----  
 0   time    object 
 1   FrL     float64
 2   FrR     float64
 3   OcR     float64
 4   marker  object 
 5   krisa   int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 2.3+ GB


In [22]:
df_filled['marker'].value_counts()

marker
none    34682226
ds      16088171
swd       836370
is        235640
Name: count, dtype: int64

In [23]:
df_filled.to_csv('data_all.csv')

In [ ]:
import pandas as pd

def process_none_marker(csv_file, output_file):
    # Загрузка данных из CSV файла
    df = pd.read_csv(csv_file)

    # Инициализация списка индексов для удаления
    to_remove = []

    # Перебор строк и поиск промежутков с "none"
    for i in range(1, len(df) - 1):
        # Проверяем, что текущий и предыдущий маркеры равны "none"
        if df['marker'][i] == 'none' and df['marker'][i-1] == 'none':
            # Найден промежуток "none", начинаем искать начало и конец
            start_idx = i - 1  # индекс начала
            end_idx = i + 1    # индекс конца
            
            # Находим все подряд идущие 'none' для начала
            while start_idx > 0 and df['marker'][start_idx-1] == 'none':
                start_idx -= 1
                
            # Находим все подряд идущие 'none' для конца
            while end_idx < len(df) and df['marker'][end_idx] == 'none':
                end_idx += 1

            # Оставляем только первые 2000 и последние 2000 значений - по 5 сек
            to_remove.extend(range(start_idx + 2000, end_idx - 2000))

    # Удаляем найденные индексы
    df.drop(to_remove, inplace=True)

    # Сохраняем измененный DataFrame в новый CSV
    df.to_csv(output_file, index=False)
    print(f"Обработка завершена. Данные сохранены в {output_file}")

# Пример использования:
process_none_marker("data_all.csv", "processed_data_all.csv")